<a href="https://colab.research.google.com/github/Saraldedv/CCADMACL_EXERCISES_COM222ML/blob/main/SARALDE_Exercise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 2: Use Gradient Boost for Regression

Instructions:

- Use the Dataset File to train your model
- Use the Test File to generate your results
- Use the Sample Submission file to generate the same format
Submit your results to:
https://www.kaggle.com/competitions/playground-series-s4e12/overview



In [49]:
import pandas as pd


from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.discriminant_analysis import StandardScaler

## Dataset
Train, test and sample submission file can be found in this link
https://www.kaggle.com/competitions/playground-series-s4e12/data

## 1. Load the Data

In [34]:
# put your answer here
sample_df = pd.read_csv('sample_submission.csv')
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')

In [36]:
X = train_df.drop("Premium Amount", axis=1)
y = train_df["Premium Amount"]

In [39]:
for col in train_df.columns:
    if col not in test_df.columns:
        print(f"Column '{col}' not found. skipping")
        continue

    if train_df[col].dtype in ["float64", "int64"]:
        train_df[col].fillna(0, inplace=True)
        test_df[col].fillna(0, inplace=True)
    elif train_df[col].dtype == "object":
        train_df[col].fillna("missing", inplace=True)
        test_df[col].fillna("missing", inplace=True)

<ipython-input-39-8901de332252>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df[col].fillna(0, inplace=True)
<ipython-input-39-8901de332252>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.metho

Column 'Premium Amount' not found. skipping


## 2. Perform Data preprocessing

In [41]:
target_column = "Premium Amount"
X = train_df.drop(columns=[target_column, 'id', 'Group', 'Year', 'Month', 'Day', 'Week'], errors='ignore')
y = train_df[target_column]

In [42]:
categorical_features = X.select_dtypes(
    include=["object"]
).columns.tolist()

numerical_features = X.select_dtypes(
    include=["float64", "int64"]
).columns.tolist()

In [43]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", StandardScaler(), numerical_features),
    ]
)

In [57]:
xgb_model = XGBRegressor(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.05,
    random_state=42,
)

## 3. Create a Pipeline

In [58]:
preprocessor = ColumnTransformer(
   transformers=[
       ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features),
       ("num", StandardScaler(), numerical_features),
   ]
)

In [59]:
pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("regressor", xgb_model),
    ]
)

## 4. Train the Model

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [61]:
pipeline.fit(X_train, y_train)

y_test_pred = pipeline.predict(X_test)

## 5. Evaluate the Model

In [62]:
val_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
print(f"Validation RMSE: {val_rmse:.4f}")

Validation RMSE: 854.0849


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Generate Submission File

Choose the model that has the best performance to generate a submission file.

In [63]:
test_features = test_df.drop(columns=['id', 'Group', 'Year', 'Month', 'Day', 'Week'], errors='ignore')
test_predictions = pipeline.predict(test_features)

submission_df = pd.DataFrame({
    "id": test_df["id"],
    "Premium Amount": test_predictions,
})

submission_df.to_csv("submission_file.csv", index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv
